In [14]:
import requests
import json
import pandas as pd
from tqdm import tqdm

In [18]:
# Riot API Key
API_KEY = "RGAPI-6e397df8-7dc5-49ec-85db-203cb6fd01b5"

GAME_NAME = "SkywalkerLin"
TAG_LINE = "na1"

# Base URLs for Riot API
ACCOUNT_URL = "https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}"
MATCHLIST_URL = "https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?count={count}"
MATCH_URL = "https://americas.api.riotgames.com/tft/match/v1/matches/{match_id}"

# Function to get Summoner PUUID
def get_summoner_puuid(game_name, tag_line):
    url = ACCOUNT_URL.format(gameName=game_name, tagLine=tag_line) + f"?api_key={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()["puuid"]
    else:
        print("Error fetching PUUID:", response.json())
        return None

# Function to fetch match IDs
def get_match_ids(puuid, count=5):
    url = MATCHLIST_URL.format(puuid=puuid, count=count) + f"&api_key={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error fetching match IDs:", response.json())
        return []

# Function to fetch match details
def get_match_details(match_id):
    url = MATCH_URL.format(match_id=match_id) + f"?api_key={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error fetching match details for", match_id, response.json())
        return None

In [20]:
# Get PUUID
puuid = get_summoner_puuid(GAME_NAME, TAG_LINE)
if puuid:
    print(f"PUUID for {GAME_NAME}#{TAG_LINE}: {puuid}")
    
    # Get Match IDs
    match_ids = get_match_ids(puuid, count=5)
    print("Match IDs:", match_ids)
    
    # Get Match Details
    match_data_list = []
    for match_id in tqdm(match_ids, desc="Fetching Matches"):
        match_data = get_match_details(match_id)
        if match_data:
            match_data_list.append(match_data)
    
    # Save raw data to JSON file
    with open("tft_matches.json", "w", encoding="utf-8") as f:
        json.dump(match_data_list, f, indent=2)
    
    print("Match data saved to 'tft_matches.json'")
    
    # Convert to Pandas DataFrame for analysis
    df = pd.json_normalize(match_data_list, record_path=['info', 'participants'])
    print(df.head())  # Display first few rows of processed data
else:
    print("Failed to retrieve PUUID. Check API Key and Riot ID.")


PUUID for SkywalkerLin#na1: rC9fkNsc87GwPDWnjQugEwcbmONnmQM7nKHt9O1thwtLKeVjRh4xMta5sUbAhFrmVIU6DVAIqMQJLw
Match IDs: ['NA1_5198806071', 'NA1_5198065321', 'NA1_5197757805', 'NA1_5196504135', 'NA1_5196494117']


Fetching Matches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.01it/s]


Match data saved to 'tft_matches.json'
   gold_left  last_round  level  placement  players_eliminated  \
0          1          31      9          4                   1   
1          1          28      8          6                   0   
2          0          28      8          7                   0   
3         14          26      8          8                   0   
4         25          33      8          3                   2   

                                               puuid riotIdGameName  \
0  xM8sDavlCWvZGttgyCg_rYl3X-JexM8p38hkgM9QJ2zKF9...         Myukii   
1  rC9fkNsc87GwPDWnjQugEwcbmONnmQM7nKHt9O1thwtLKe...   SkywalkerLin   
2  G9dj93rIWK1AjZkpcNfOfxUkm15Zr324RbJbRZMIiZoX6i...           Regi   
3  Rwmq5oBepy7RpkHJr1AEiTC0CSP0XvuCSvaFZ-557LwRRa...   minimalnebby   
4  AQsAcU398Spn-88O743J35zpehFh94F_pIx6bD6GQz7A4b...      adcqueenn   

  riotIdTagline  time_eliminated  total_damage_to_players  \
0           NA1      1780.857422                       67   
1           NA1